In [1]:
import imp
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

C:\Users\TAM\AppData\Local\Temp\ipykernel_13100\2202898749.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Qty 4 Months/02 - Back Order 2022 VAPC - 2022 24 08 - CHECKED.xlsx")

In [3]:
data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)
data_nhapxuatton=data_nhapxuatton.rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6)
data_banhang=data_banhang.rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4)
data_giaban=data_giaban.rename(columns={"Mã số":"ID"}).dropna(subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M")
data_danhmuchanghoa=data_danhmuchanghoa.rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_13100\2138151211.py:1: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)


In [ ]:
# Giá vốn
data_nhapxuatton

In [4]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [6]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID').merge(data_giaban,on="ID")

# TÍnh số tháng tồn trong kho
data_nhap_ban["Số tháng tồn trong kho"]=(363/30)/data_nhap_ban["Giá vốn"]/(np.where(
    # Nếu Tồn dương và đầu kỳ dương thì lấy Giá vốn / (TB Giá trị đầu kỳ + Cuối kỳ)
                    (data_nhap_ban["Giá trị đầu kỳ"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                               np.mean([data_nhap_ban["Tổng đầu kỳ"],data_nhap_ban["Tổng tồn"]])/
                                np.mean([data_nhap_ban["Giá trị đầu kỳ"],data_nhap_ban["Giá trị tồn"]]),-999))

# Show kết quả
data_nhap_ban_filter=data_nhap_ban[["ID","Tên vật tư","Hãng","Giá vốn","Giá bán","Tổng đầu kỳ","Tổng nhập","Tổng xuất","Tổng tồn", "Giá trị đầu kỳ", "Giá trị nhập" ,"Giá trị xuất", "Giá trị tồn","Số tháng tồn trong kho"]]
data_nhap_ban_filter=data_nhap_ban_filter.query("`Giá vốn`>0 & `Tổng tồn` >0 ").sort_values(by=["Số tháng tồn trong kho"], ascending=False).reset_index(drop=True)

KeyError: 'Giá vốn'

In [ ]:
data_MITSUBISHI_6M=data_nhap_ban_filter.query("`Hãng` == 'MITSUBISHI' & `Số tháng tồn trong kho`>6").sort_values(by="Số tháng tồn trong kho",ascending=False).reset_index(drop=True)

In [ ]:
data_MITSUBISHI_6M

In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Lọc tồn theo hãng/Export.xlsx') as writer:
    # data_mitsubishi.to_excel(writer, sheet_name='MITSUBISHI')
    data_nhap_ban.to_excel(writer,sheet_name="Nhap Ban")
    data_nhap_ban_filter.to_excel(writer,sheet_name="Filter")
    data_MITSUBISHI_6M.to_excel(writer,sheet_name="MITSUBISHI_6Thang")